In [1]:
#!for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done
import tensorflow as tf
import os
from data_processing import dataset
from models import model
from utils import tools
from plots import plots, edge_detection_plots

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
#np.set_printoptions(threshold=sys.maxsize)

2022-12-01 11:22:06.335293: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 11:22:06.466989: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-01 11:22:06.498204: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-01 11:22:07.132623: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
config_directory = 'edge_detection_without_prior'

config_path = os.path.join(os.getcwd(), 'configs', config_directory)
Model = model.Model(config_path)
DataProcessing = dataset.DataProcessing(Model.cfg["INPUT_SHAPE"], Model.cfg["OUTPUT_SHAPE"], config_path)
tools.parser(Model.cfg, DataProcessing.cfg)
Model.load_data(DataProcessing.cfg["NAME"])
DataProcessing.path_definitions()

2022-12-01 11:22:08.702571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-01 11:22:08.709510: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-01 11:22:08.709665: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-01 11:22:08.710234: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

# Load Dataset, Preprocess Images and Dataset

In [3]:
if Model.train_model:
    train_ds, img_count_train = DataProcessing.load_dataset(DataProcessing.key.train)
test_ds, img_count_test = DataProcessing.load_dataset(DataProcessing.key.test)

img_only_ds, img_count_img_only_ds = DataProcessing.load_dataset(DataProcessing.key.img_only)

#if Model.train_model:
#    for inp, out in train_ds.take(1):
#        edge_detection_plots.plot_edges(images=inp['in_img'], labels=out['out_edge'],
#                                        batch_size=DataProcessing.cfg['TEST']['BATCH_SIZE'],
#                                        num_classes=DataProcessing.num_classes['edge'])

The TRAIN DS contains 1500 images.
The TEST DS contains 300 images.
The IMG_ONLY DS contains 80 images.


# Define, Compile and Train Model

In [ ]:
if Model.train_model:
    model = Model.get_neural_network_model(DataProcessing.num_classes['edge'])
    # model.summary()

    lr = Model.get_lr(img_count_train, DataProcessing.cfg['TRAIN']['BATCH_SIZE'])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr),
                  loss=Model.get_loss_function(),
                  metrics=Model.get_metrics(DataProcessing.num_classes['edge']))

    history = model.fit(train_ds, epochs=Model.cfg["EPOCHS"], validation_data=train_ds,
                        callbacks=Model.get_callbacks(), verbose=1)

model = Model.get_best_model_from_checkpoints()

(640, 360, 3)
Epoch 1/60


2022-12-01 11:22:26.650788: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 1021 of 1500
2022-12-01 11:22:30.826278: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.
2022-12-01 11:22:31.762860: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-12-01 11:22:32.912975: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


500/500 [==============================] - 79s 116ms/step - loss: 170.8565 - accuracy: 0.9609 - accuracy_1: 0.9176 - accuracy_2: 0.9641 - accuracy_3: 0.9663 - accuracy_4: 0.9761 - accuracy_5: 0.9805 - f1: 0.0141 - precision: 0.0129 - recall: 0.0260 - f1_1: 0.0062 - precision_1: 0.0035 - recall_1: 0.0589 - f1_2: 0.0330 - precision_2: 0.0961 - recall_2: 0.0245 - f1_3: 0.0056 - precision_3: 0.0051 - recall_3: 0.0136 - f1_4: 0.0063 - precision_4: 0.0041 - recall_4: 0.0319 - f1_5: 0.0050 - precision_5: 0.0037 - recall_5: 0.0181 - val_loss: 82.0256 - val_accuracy: 0.9914 - val_accuracy_1: 0.9952 - val_accuracy_2: 0.9765 - val_accuracy_3: 0.9911 - val_accuracy_4: 0.9973 - val_accuracy_5: 0.9968 - val_f1: 0.0748 - val_precision: 0.5679 - val_recall: 0.0401 - val_f1_1: 0.0000e+00 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_f1_2: 0.1282 - val_precision_2: 0.5737 - val_recall_2: 0.0721 - val_f1_3: 0.0033 - val_precision_3: 0.3120 - val_recall_3: 0.0017 - val_f1_4: 1.4597e-04 - 

500/500 [==============================] - 56s 111ms/step - loss: 51.5671 - accuracy: 0.9927 - accuracy_1: 0.9952 - accuracy_2: 0.9822 - accuracy_3: 0.9923 - accuracy_4: 0.9973 - accuracy_5: 0.9968 - f1: 0.4373 - precision: 0.6751 - recall: 0.3241 - f1_1: 0.0789 - precision_1: 0.4270 - recall_1: 0.0438 - f1_2: 0.5551 - precision_2: 0.7095 - recall_2: 0.4578 - f1_3: 0.4026 - precision_3: 0.6153 - recall_3: 0.3003 - f1_4: 0.1016 - precision_4: 0.3292 - recall_4: 0.0606 - f1_5: 0.0000e+00 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - val_loss: 56.9700 - val_accuracy: 0.9912 - val_accuracy_1: 0.9952 - val_accuracy_2: 0.9748 - val_accuracy_3: 0.9920 - val_accuracy_4: 0.9973 - val_accuracy_5: 0.9968 - val_f1: 0.4688 - val_precision: 0.4964 - val_recall: 0.4441 - val_f1_1: 0.0360 - val_precision_1: 0.5578 - val_recall_1: 0.0186 - val_f1_2: 0.5442 - val_precision_2: 0.4808 - val_recall_2: 0.6269 - val_f1_3: 0.4794 - val_precision_3: 0.5708 - val_recall_3: 0.4131 - val_f1_4: 0.2866 - val_p

INFO:tensorflow:Assets written to: /home/david/SemesterProject/Models/edge_detection/edge_detection_without_prior/CKPT/ckpt-loss=47.72-epoch=10.00-f1=0.4964/assets


...

500/500 [==============================] - 58s 115ms/step - loss: 41.4079 - accuracy: 0.9942 - accuracy_1: 0.9957 - accuracy_2: 0.9864 - accuracy_3: 0.9937 - accuracy_4: 0.9975 - accuracy_5: 0.9975 - f1: 0.5760 - precision: 0.7120 - recall: 0.4839 - f1_1: 0.3164 - precision_1: 0.6012 - recall_1: 0.2157 - f1_2: 0.6572 - precision_2: 0.7507 - recall_2: 0.5850 - f1_3: 0.5351 - precision_3: 0.6992 - recall_3: 0.4337 - f1_4: 0.3003 - precision_4: 0.6095 - recall_4: 0.2001 - f1_5: 0.5588 - precision_5: 0.5879 - recall_5: 0.5334 - val_loss: 40.7535 - val_accuracy: 0.9937 - val_accuracy_1: 0.9958 - val_accuracy_2: 0.9845 - val_accuracy_3: 0.9938 - val_accuracy_4: 0.9977 - val_accuracy_5: 0.9964 - val_f1: 0.5416 - val_precision: 0.7319 - val_recall: 0.4299 - val_f1_1: 0.4279 - val_precision_1: 0.6152 - val_recall_1: 0.3280 - val_f1_2: 0.5607 - val_precision_2: 0.8807 - val_recall_2: 0.4113 - val_f1_3: 0.5714 - val_precision_3: 0.7542 - val_recall_3: 0.4599 - val_f1_4: 0.3677 - val_precision_4: 

INFO:tensorflow:Assets written to: /home/david/SemesterProject/Models/edge_detection/edge_detection_without_prior/CKPT/ckpt-loss=38.65-epoch=20.00-f1=0.6380/assets


INFO:tensorflow:Assets written to: /home/david/SemesterProject/Models/edge_detection/edge_detection_without_prior/CKPT/ckpt-loss=38.65-epoch=20.00-f1=0.6380/assets


500/500 [==============================] - 90s 180ms/step - loss: 39.0756 - accuracy: 0.9941 - accuracy_1: 0.9957 - accuracy_2: 0.9861 - accuracy_3: 0.9937 - accuracy_4: 0.9976 - accuracy_5: 0.9975 - f1: 0.6050 - precision: 0.7273 - recall: 0.5181 - f1_1: 0.3962 - precision_1: 0.6655 - recall_1: 0.2823 - f1_2: 0.6769 - precision_2: 0.7629 - recall_2: 0.6084 - f1_3: 0.5653 - precision_3: 0.7011 - recall_3: 0.4742 - f1_4: 0.3710 - precision_4: 0.6411 - recall_4: 0.2614 - f1_5: 0.5819 - precision_5: 0.6197 - recall_5: 0.5489 - val_loss: 38.6482 - val_accuracy: 0.9940 - val_accuracy_1: 0.9961 - val_accuracy_2: 0.9858 - val_accuracy_3: 0.9938 - val_accuracy_4: 0.9977 - val_accuracy_5: 0.9964 - val_f1: 0.6380 - val_precision: 0.6680 - val_recall: 0.6106 - val_f1_1: 0.4288 - val_precision_1: 0.7011 - val_recall_1: 0.3089 - val_f1_2: 0.7087 - val_precision_2: 0.6994 - val_recall_2: 0.7182 - val_f1_3: 0.5991 - val_precision_3: 0.7116 - val_recall_3: 0.5173 - val_f1_4: 0.3927 - val_precision_4: 

500/500 [==============================] - 97s 193ms/step - loss: 36.7082 - accuracy: 0.9943 - accuracy_1: 0.9957 - accuracy_2: 0.9869 - accuracy_3: 0.9938 - accuracy_4: 0.9977 - accuracy_5: 0.9976 - f1: 0.6263 - precision: 0.7346 - recall: 0.5459 - f1_1: 0.3900 - precision_1: 0.6191 - recall_1: 0.2856 - f1_2: 0.7001 - precision_2: 0.7772 - recall_2: 0.6370 - f1_3: 0.5850 - precision_3: 0.7142 - recall_3: 0.4964 - f1_4: 0.4102 - precision_4: 0.6521 - recall_4: 0.3000 - f1_5: 0.6063 - precision_5: 0.6178 - recall_5: 0.5956 - val_loss: 35.6151 - val_accuracy: 0.9945 - val_accuracy_1: 0.9962 - val_accuracy_2: 0.9873 - val_accuracy_3: 0.9938 - val_accuracy_4: 0.9978 - val_accuracy_5: 0.9974 - val_f1: 0.6606 - val_precision: 0.7147 - val_recall: 0.6141 - val_f1_1: 0.4132 - val_precision_1: 0.7859 - val_recall_1: 0.2803 - val_f1_2: 0.7231 - val_precision_2: 0.7559 - val_recall_2: 0.6931 - val_f1_3: 0.6396 - val_precision_3: 0.6685 - val_recall_3: 0.6130 - val_f1_4: 0.4758 - val_precision_4: 

# Plot Results

In [ ]:
if Model.train_model:
    plot_losses = ["loss", "loss"]
    plot_metrics = ["accuracy", "f1", "recall", "precision"]

    path = os.path.join(Model.Data.paths["FIGURES"], "training.svg")

    plots.plot_training_history(history=history.history, list_of_loss_names=plot_losses,
                                list_of_metric_names=plot_metrics,
                                save=Model.cfg["SAVE"], path=path)

In [ ]:
for inp, out in test_ds.take(1):
    pred = model.predict(inp)
    edge_detection_plots.plot_edges(images=inp['in_img'], labels=out['out_edge'],
                                    predictions=pred,
                                    batch_size=DataProcessing.cfg['TEST']['BATCH_SIZE'],
                                    num_classes=DataProcessing.num_classes['edge'])

In [ ]:
for inp, out in img_only_ds.take(1):
    pred = model.predict(inp)
    edge_detection_plots.plot_edges(images=inp['in_img'],
                                    predictions=pred,
                                    batch_size=DataProcessing.cfg['IMG_ONLY']['BATCH_SIZE'],
                                    num_classes=DataProcessing.num_classes['edge'])

In [ ]:
### Maximum F1 Score:
path_metrics_evaluation_plot = os.path.join(Model.Data.paths["FIGURES"],
                                            "threshold_metrics_evaluation_test_ds.svg")

edge_detection_plots.plot_threshold_metrics_evaluation(model=model, ds=test_ds,
                                                       num_classes=DataProcessing.num_classes['edge'],
                                                       classes_displayed_individually=True,
                                                       save=Model.cfg["SAVE"],
                                                       path=path_metrics_evaluation_plot,
                                                       accuracy_y_lim_min=0.8)


In [ ]:
if Model.cfg["CONVERT_TO_TFLITE"]:
    Model.convert_model_to_tflite(model)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(include_top=False,input_shape=[640,360,3])

input_model = tf.keras.Input(shape=[640,360,3])
x = tf.keras.applications.mobilenet_v2.preprocess_input(input_model)
x = base_model(x, training=True)
model = tf.keras.Model(input_model, x)
model.trainable = False
model.save(Model.Data.paths['TFLITE'])

converter = tf.lite.TFLiteConverter.from_saved_model(Model.Data.paths["TFLITE"])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model, gpu_compatibility=True)

# Save the model.
with open(Model.Data.files['OUTPUT_TFLITE_MODEL'], 'wb') as f:
    f.write(tflite_model)